In [18]:
import pandas as pd
import requests
import time

# Reddit API credentials
client_id = 'Nrbc6n5KVQ6fEQ_7JUfH3A'
client_secret = 'Y0STrnfxdJh_r5h9TS5qxx2tAs5swg'
username = 'Important_Trade_7759'
password = 'saima123'
user_agent = 'data'

# Authentication with Reddit API
auth = requests.auth.HTTPBasicAuth(client_id, client_secret)
data = {'grant_type': 'password', 'username': username, 'password': password}
headers = {'User-Agent': user_agent}

res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
token = res.json()['access_token']
headers['Authorization'] = f'bearer {token}'

# Generalized function to fetch data from Reddit
def fetch_data(url, params=None):
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data from {url}: {response.status_code}")
        return None

# Function to collect posts from a subreddit with pagination
def collect_posts_by_category(subreddit, category, limit=1000):
    posts = []
    after = None  # To keep track of pagination
    collected_posts = 0  # Counter for posts collected so far

    # Set the correct endpoint based on the category
    url = f'https://oauth.reddit.com/r/{subreddit}/{category}'

    while collected_posts < limit:
        params = {'limit': 100}  # Max limit per request
        if after:
            params['after'] = after

        data = fetch_data(url, params)

        if data and 'data' in data:
            for post in data['data']['children']:
                post_id = post['data']['id']
                post_title = post['data']['title']
                post_content = post['data']['selftext']
                post_created_utc = post['data']['created_utc']
                post_subreddit = subreddit

                posts.append([post_title, post_content, post_created_utc, post_subreddit, category])
                collected_posts += 1

                # Stop if we reach the desired limit
                if collected_posts >= limit:
                    break

            # Update 'after' for pagination
            after = data['data']['after']
            if not after:  # No more posts to fetch
                break

            # Add a delay to avoid hitting rate limits
            time.sleep(2)  # Sleep for 2 seconds between requests

        else:
            break

    return posts[:limit]  # Return only up to the specified limit

# Function to collect posts for multiple subreddits and categories
def collect_posts_for_subreddits(subreddits, categories, limit_per_category=1000):
    all_posts = []
    for subreddit in subreddits:
        for category in categories:
            print(f"Collecting posts from r/{subreddit} in category '{category}'...")
            posts = collect_posts_by_category(subreddit, category, limit_per_category)
            all_posts.extend(posts)
    return all_posts

# List of subreddits and categories to collect data from
subreddits_to_collect = ['technology', 'gadgets']
categories_to_collect = ['new', 'hot']

# Collect data from specified subreddits and categories
all_posts = collect_posts_for_subreddits(subreddits_to_collect, categories_to_collect, limit_per_category=1100)

# Create DataFrame
columns = ['title', 'content', 'timestamp', 'subreddit', 'category']
df_combined = pd.DataFrame(all_posts, columns=columns)

# Convert timestamp to datetime
df_combined['timestamp'] = pd.to_datetime(df_combined['timestamp'], unit='s')

# Check for unique posts based on 'title', 'content', and 'timestamp'
df_unique = df_combined.drop_duplicates(subset=['title', 'content', 'timestamp'])

# Save the unique DataFrame to a CSV file
df_unique.to_csv('./data/reddit_posts_new_hot.csv', index=False)

# Output the unique DataFrame (for checking)
print(df_unique.head())
print(f"Total unique posts: {df_unique.shape[0]}")

                                               title content  \
0  Brightness of first Chinese broadband constell...           
1  Collapse of national security elites’ cyber fi...           
2  Teens already on social media could be exempt ...           
3  License Plate Readers Are Creating a US-Wide D...           
4  Japan’s tokamak sets world record, achieves pl...           

            timestamp   subreddit category  
0 2024-10-06 05:25:43  technology      new  
1 2024-10-06 04:46:03  technology      new  
2 2024-10-06 03:55:20  technology      new  
3 2024-10-06 03:18:42  technology      new  
4 2024-10-06 02:28:05  technology      new  
Total unique posts: 1661
